# Assignment 3

## 1. 
Write a Python function that takes as input a DNA sequence string and returns its reverse complement string. For example, given an input string “CATGCCGGAATT”, the function returns the reverse complement string “AATTCCGGCATG” [2pt].

In [1]:
def reverseComplement1(seq):
    bp = {"A": "T", "T": "A", "C": "G", "G": "C"}
    revseq = ""
    for i in range(len(seq) - 1, -1, -1):
        revseq += bp[seq[i]] # You can also use "if" statements to assign bases
    return revseq
        
def reverseComplement2(seq):
    bp = {"A": "T", "T": "A", "C": "G", "G": "C"}
    return "".join(map(lambda x: bp[x], seq[::-1]))

In [2]:
print(reverseComplement1("CATGCCGGAATT"))
print(reverseComplement2("CATGCCGGAATT"))

AATTCCGGCATG
AATTCCGGCATG


## 2. 
Write a Python function (or functions) to find all ORFs from six different reading frames (+1, +2, +3, -1, -2, -3) in a sequence string (you may need function #1 to get reverse complement string). Since we are only interested in relatively long ORFs, the function should only return those ORFs longer than an input threshold value. Compute the length of the ORFs as the number of codons (counting the start and stop codon). The start codon is “ATG” and the stop codons are “TAG”, “TAA”, and “TGA” [2pt].

In the example below, there are two overlapping ORFs (the first three lines are the same sequence and the last three lines are the same reverse complement sequence, with the commas in different places to make it easier to read). The length of the ORF on the first line is seven. The length of the ORF on the second line is six. Each ORF starts with an “ATG”, ends up with a stop codon, and there is no in-frame stop codon in the sequence.


The inputs to the function should be the sequence string and the threshold value. The output should be a list of lists. The length of the list should be the number of ORFs. Each element of the list should be another list with three numbers describing an individual ORF: the frame number, the nucleotide position of the first position of the start codon, and the nucleotide position of the first position of the in-frame stop codon. For the example above, with threshold 5, the output should be: [[1,9,27],[2,1,16]].


In [3]:
# Brute force
def findORF_bruteforce(seq, startPos, threshold):
    #startPos = 0, 1, 2
    stopcodons = {"TAG", "TAA", "TGA"}
    result = []
    for i in range(startPos, len(seq), 3):
        subseq = seq[i:i+3]
        if subseq == "ATG":
            for j in range(i + 3, len(seq), 3):
                if seq[j:j+3] in stopcodons:
                    if (j - i) / 3 >= threshold:
                        result.append([startPos+1, i, j])
                    break
    return result

def findORFs_bf(seq, threshold):
    result = []
    for i in range(3):
        subresult = findORF_bruteforce(seq, i, threshold)
        result = result + subresult
    revseq = reverseComplement2(seq)
    for i in range(3):
        subresult = findORF_bruteforce(revseq, i, threshold)
        for j in range(len(subresult)):
            subresult[j][0] = -subresult[j][0]
        result = result + subresult
    return result

In [4]:
# An ORF can only contain one stop codon (in the end). 
# An ORF can have many start codons inside. 
## => Search the sequence to find all start codons and the first stop codon after them. 
def findORF(seq, threshold):
    startCodon = "ATG"
    stopCodons = {"TAG", "TAA", "TGA"} #Use set to directly match if a 3-mer is a stop codon
    startPositions = {1: [], 2: [], 3: []} #You can also init it as a list: [[], [], []] with index 0,1,2 as frame 1,2,3
    #startPositions = {1: collections.deque(), 2: [], 3: []}
    #GATGGGGGGGCCCTAG
    result = []
    
    for i in range(len(seq)-3):
        frame = i % 3 + 1
        if seq[i:i+3] == startCodon:
            startPositions[frame].append(i)
        elif seq[i:i+3] in stopCodons:
            #A more neat way to the part below is to use collections.deque() to define values in startPositions
            #while len(startPositions[frame]) != 0:
            #    i = startPositions[frame].popleft()
            for startPos in startPositions[frame]:
                if (i - startPos) // 3 >= threshold:
                    result.append([frame, startPos, i])
                else:
                    break
            startPositions[frame] = []
            
    return result
            

def findORF_withRev(seq, threshold):
    result = findORF(seq, threshold)
    revResult = findORF(reverseComplement2(seq), threshold)
    for i in range(len(revResult)):
        revResult[i][0] = -revResult[i][0]
    return result + revResult

In [6]:
print(findORF_withRev("AATGCCCAAATGCTTTTAAAACCCATGTAGC", 5))
print(findORFs_bf("AATGCCCAAATGCTTTTAAAACCCATGTAGC", 5))



[[2, 1, 16], [1, 9, 27]]
[[1, 9, 27], [2, 1, 16]]


## 3. 
Download the FASTA file “ACE2.fasta” from Blackboard. Use function #2, with threshold 700, to find all ORFs. [2pt].

In [7]:
#We use the same function in Assignment 1 to read fasta file.

def readFASTA(filename):
    seq = ""
    with open(filename, "r") as fin: #open the file object as fin
        for line in fin: 
            #iterate each line in the file
            if line[0] == ">": 
                #if the line starts with a ">" then it means it is the description
                continue 
                # we move to the next line
                
            #you can also skip the first line and read lines afterwards
            seq += line.strip() # we add this line to the sequence string with "\n" trimmed
            
    return seq

In [8]:
seq = readFASTA("ACE2.fasta")

In [16]:
print(findORF_withRev(seq, 700))

[[2, 49, 2464], [2, 232, 2464], [2, 292, 2464]]


## 4.
Write a Python function that takes two inputs: a float gc and an integer l. This function returns the expected value E[X] and the probability P(X>L), where X is a geometric random variable representing the number of codons until getting a stop codon [2pt].

For example, with the inputs (gc=0.4, I=50):

First, the function calculates p, the probability of getting a stop codon:

𝑝 = 𝑃(𝑇𝐴𝐺 𝑜𝑟 𝑇𝐴𝐴 𝑜𝑟 𝑇𝐺𝐴) = (0.3 × 0.3 × 0.2) + (0.3 × 0.3 × 0.3) + (0.3 × 0.2 × 0.3) = 0.063

Second, the function calculates 𝐸[𝑋] = 1⁄𝑝 = 15.873

Third, the function calculates 𝑃(𝑋 > 𝐿) = 1 − 𝑃(𝑋 ≤ 50) = 1 − 0.961 = 0.039 Finally, the function returns 15.873 and 0.039

In [13]:
from scipy.stats import geom
def calc(gcContent, L):
    prob_AT = (1 - gcContent) / 2
    prob_CG = gcContent / 2
    p = prob_AT*prob_AT*prob_CG + prob_AT*prob_AT*prob_AT + prob_AT*prob_CG*prob_AT
    x = geom(p)
    return x.mean(), 1 - x.cdf(L)

In [14]:
print(calc(0.4, 50))

(15.873015873015873, 0.03863487792946596)
